# Introduction  
This notebook provide 4 models and 1 optimizer.  
They help you when you build first model.  
Quick simple model is the best choice in the beginning pariod.  

# Import packages

In [201]:
from tqdm import tqdm
from collections import Counter
import lightgbm as lgb
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from xgboost import plot_importance
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import cohen_kappa_score, mean_squared_error, f1_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import json
import pandas as pd
import numpy as np
import warnings
import random
import time
import os
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Lock, Process, Queue, current_process
import scipy as sp
from functools import partial
from numba import jit
from sklearn.preprocessing import OneHotEncoder
pd.set_option('display.max_columns', 1000)
warnings.filterwarnings('ignore')

In [137]:
#load data
train = pd.read_csv("train.csv")

test_df =pd.read_csv("test.csv")
#trian=train.drop(columns=['open_channels'])
train_df=train.drop(columns=['time'])

# Prepare helper functions

In [184]:
def lgb_Metric(y_true, y_pred):
    #labels = dtrain.get_label()
    #print(preds.shape)
    #print(preds)
    y_pred = np.round(np.clip(y_pred, 0, 10)).astype(int)#np.argmax(y_pred, axis=0)
    y_pred = np.array(y_pred).reshape(y_true.shape)
#     score = metrics.cohen_kappa_score(labels, preds, weights = 'quadratic')
    score = f1_score(y_true=y_true ,y_pred=y_pred, average='macro')
    return ('KaggleMetric', score, True)

# Define models

In [186]:
class Base_Model(object):
    
    def __init__(self, train_df, test_df, features, categoricals=[], n_splits=5, verbose=True,ps={}):
        self.train_df = train_df
        self.test_df = test_df
        self.features = features
        self.n_splits = n_splits
        self.categoricals = categoricals
        self.target = 'open_channels' #set your objective variable here
        self.cv = self.get_cv()
        self.verbose = verbose
#         self.params = self.get_params()
        self.params = self.set_params(ps)
        self.y_pred, self.score, self.model = self.fit()
        
    def train_model(self, train_set, val_set):
        raise NotImplementedError
        
    #split data x and y, and kfold
    def get_cv(self):
        #cv = GroupKFold(n_splits=self.n_splits)
        cv = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        #cv = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        return cv.split(self.train_df, self.train_df[self.target])
    
    def get_params(self):
        raise NotImplementedError
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        raise NotImplementedError
        
    def convert_x(self, x):
        return x
        
    def fit(self):
        oof_pred = np.zeros((len(train_df), ))
        y_pred = np.zeros((len(test_df), ))
        for fold, (train_idx, val_idx) in enumerate(self.cv):
            x_train, x_val = self.train_df[self.features].iloc[train_idx], self.train_df[self.features].iloc[val_idx]
            y_train, y_val = self.train_df[self.target][train_idx], self.train_df[self.target][val_idx]
            train_set, val_set = self.convert_dataset(x_train, y_train, x_val, y_val)
            model = self.train_model(train_set, val_set)
            conv_x_val = self.convert_x(x_val)
            oof_pred[val_idx] = model.predict(conv_x_val).reshape(oof_pred[val_idx].shape)
            x_test = self.convert_x(self.test_df[self.features])
            y_pred += model.predict(x_test).reshape(y_pred.shape) / self.n_splits
            print('Partial score of fold {} is: {}'.format(fold, lgb_Metric(y_val, oof_pred[val_idx])[1]))
        _, loss_score, _ = lgb_Metric(self.train_df[self.target], oof_pred)
        if self.verbose:
            print('Our macroF1 is: ', loss_score)
        return y_pred, loss_score, model

In [187]:
class Lgb_Model(Base_Model):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        return lgb.train(self.params, train_set, valid_sets=[train_set, val_set], verbose_eval=verbosity)
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
        x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]
        train_set = lgb.Dataset(x_train, y_train, categorical_feature=self.categoricals)
        val_set = lgb.Dataset(x_val, y_val, categorical_feature=self.categoricals)
        return train_set, val_set
        
    def get_params(self):
        params = {'n_estimators':5000,
                    'boosting_type': 'gbdt',
                    'objective': 'regression',
                    'metric': 'rmse',
                    'subsample': 0.75,
                    'subsample_freq': 1,
                    'learning_rate': 0.01,
                    'feature_fraction': 0.9,
                    'max_depth': 15,
                    'lambda_l1': 1,  
                    'lambda_l2': 1,
                    'early_stopping_rounds': 100
                    }
        return params
    def set_params(self,ps={}):
        params = self.get_params()
        if 'subsample_freq' in ps:
            params['subsample_freq']=int(ps['subsample_freq'])
            params['learning_rate']=ps['learning_rate']
            params['feature_fraction']=ps['feature_fraction']
            params['lambda_l1']=ps['lambda_l1']
            params['lambda_l2']=ps['lambda_l2']
            params['max_depth']=int(ps['max_depth'])
        
        return params

In [188]:
class Xgb_Model(Base_Model):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        return xgb.train(self.params, train_set, 
                         num_boost_round=5000, evals=[(train_set, 'train'), (val_set, 'val')], 
                         verbose_eval=verbosity, early_stopping_rounds=100)
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = xgb.DMatrix(x_train, y_train)
        val_set = xgb.DMatrix(x_val, y_val)
        return train_set, val_set
    
    def convert_x(self, x):
        return xgb.DMatrix(x)
        
    def get_params(self):
        params = {'colsample_bytree': 0.8,                 
            'learning_rate': 0.01,
            'max_depth': 10,
            'subsample': 1,
            'objective':'reg:squarederror',
            #'eval_metric':'rmse',
            'min_child_weight':3,
            'gamma':0.25,
            'n_estimators':5000}
        return params
    
    def set_params(self, ps={}):
        return self.get_params()

In [189]:
class Catb_Model(Base_Model):
    
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        clf = CatBoostRegressor(**self.params)
        clf.fit(train_set['X'], 
                train_set['y'], 
                eval_set=(val_set['X'], val_set['y']),
                verbose=verbosity, 
                cat_features=self.categoricals)
        return clf
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = {'X': x_train, 'y': y_train}
        val_set = {'X': x_val, 'y': y_val}
        return train_set, val_set
        
    def get_params(self):
        params = {'loss_function': 'RMSE',
                   'task_type': "CPU",
                   'iterations': 5000,
                   'od_type': "Iter",
                    'depth': 10,
                  'colsample_bylevel': 0.5, 
                   'early_stopping_rounds': 300,
                    'l2_leaf_reg': 18,
                   'random_seed': 42,
                    'use_best_model': True
                    }
        return params
    
    def set_params(self, ps={}):
        return self.get_params()

In [190]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

class Nn_Model(Base_Model):
    
    def __init__(self, train_df, test_df, features, categoricals=[], n_splits=5, verbose=True):
        features = features.copy()
        if len(categoricals) > 0:
            for cat in categoricals:
                enc = OneHotEncoder()
                train_cats = enc.fit_transform(train_df[[cat]])
                test_cats = enc.transform(test_df[[cat]])
                cat_cols = ['{}_{}'.format(cat, str(col)) for col in enc.active_features_]
                features += cat_cols
                train_cats = pd.DataFrame(train_cats.toarray(), columns=cat_cols)
                test_cats = pd.DataFrame(test_cats.toarray(), columns=cat_cols)
                train_df = pd.concat([train_df, train_cats], axis=1)
                test_df = pd.concat([test_df, test_cats], axis=1)
        scalar = MinMaxScaler()
        train_df[features] = scalar.fit_transform(train_df[features])
        test_df[features] = scalar.transform(test_df[features])
        print(train_df[features].shape)
        super().__init__(train_df, test_df, features, categoricals, n_splits, verbose)
        
    def train_model(self, train_set, val_set):
        verbosity = 100 if self.verbose else 0
        model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(train_set['X'].shape[1],)),
            tf.keras.layers.Dense(200, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(100, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(50, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(25, activation='relu'),
            tf.keras.layers.LayerNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(1, activation='relu')
        ])
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-4), loss='mse')
        print(model.summary())
        save_best = tf.keras.callbacks.ModelCheckpoint('nn_model.w8', save_weights_only=True, save_best_only=True, verbose=1)
        early_stop = tf.keras.callbacks.EarlyStopping(patience=20)
        model.fit(train_set['X'], 
                train_set['y'], 
                validation_data=(val_set['X'], val_set['y']),
                epochs=100,
                 callbacks=[save_best, early_stop])
        model.load_weights('nn_model.w8')
        return model
        
    def convert_dataset(self, x_train, y_train, x_val, y_val):
        train_set = {'X': x_train, 'y': y_train}
        val_set = {'X': x_val, 'y': y_val}
        return train_set, val_set
        
    def get_params(self):
        return None
    def set_params(self, ps={}):
        return self.get_params()

In [191]:
#Optimizer
from bayes_opt import BayesianOptimization

def LGB_Beyes(subsample_freq,
                    learning_rate,
                    feature_fraction,
                    max_depth,
                    lambda_l1,
                    lambda_l2):
    params={}
    params['subsample_freq']=subsample_freq
    params['learning_rate']=learning_rate
    params['feature_fraction']=feature_fraction
    params['lambda_l1']=lambda_l1
    params['lambda_l2']=lambda_l2
    params['max_depth']=max_depth
    lgb_model= Lgb_Model(train, test, features, categoricals=categoricals,ps=params)
    print('kappa: ',lgb_model.score)
    return lgb_model.score

bounds_LGB = {
    'subsample_freq': (1, 10),
    'learning_rate': (0.025, 0.5),
    'feature_fraction': (0.5, 1),
    'lambda_l1': (0, 5),
    'lambda_l2': (0, 5),
    'max_depth': (13, 17),
}

LGB_BO = BayesianOptimization(LGB_Beyes, bounds_LGB, random_state=1029)
import warnings
init_points = 16
n_iter = 16
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    LGB_BO.maximize(init_points=init_points, n_iter=n_iter, acq='ucb', xi=0.0, alpha=1e-6)

ModuleNotFoundError: No module named 'bayes_opt'

# Training Models

In [192]:
categoricals = [] #In this time, there were no categorical feature
features = test.columns

In [193]:
lgb_model = Lgb_Model(train_df, test, features, categoricals=categoricals)
#if you want to use hyperparameter optimizer, please use following code
#lgb_model = Lgb_Model(train, test, features, categoricals=categoricals, ps=LGB_BO.max['params']) 

Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.606499	valid_1's rmse: 0.613267
[200]	training's rmse: 0.336658	valid_1's rmse: 0.35871
[300]	training's rmse: 0.272153	valid_1's rmse: 0.308824
[400]	training's rmse: 0.25549	valid_1's rmse: 0.301337
[500]	training's rmse: 0.247544	valid_1's rmse: 0.300259
[600]	training's rmse: 0.241502	valid_1's rmse: 0.300217
Early stopping, best iteration is:
[527]	training's rmse: 0.245759	valid_1's rmse: 0.300187
Partial score of fold 0 is: 0.7645927792705627
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 0.606852	valid_1's rmse: 0.612855
[200]	training's rmse: 0.336507	valid_1's rmse: 0.35682
[300]	training's rmse: 0.271762	valid_1's rmse: 0.306338
[400]	training's rmse: 0.254822	valid_1's rmse: 0.299649
[500]	training's rmse: 0.246473	valid_1's rmse: 0.299391
Early stopping, best iteration is:
[481]	training's rmse: 0.247824	valid_1's rmse: 0.299305
Partial score of fold

In [194]:
xgb_model = Xgb_Model(train, test, features, categoricals=categoricals)

[0]	train-rmse:6.80254	val-rmse:6.81098
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[100]	train-rmse:2.5085	val-rmse:2.514
[200]	train-rmse:0.95374	val-rmse:0.965007
[300]	train-rmse:0.407725	val-rmse:0.453076
[400]	train-rmse:0.234153	val-rmse:0.325855
[500]	train-rmse:0.188032	val-rmse:0.304549
[600]	train-rmse:0.174816	val-rmse:0.301674
[700]	train-rmse:0.170768	val-rmse:0.301402
Stopping. Best iteration:
[680]	train-rmse:0.171333	val-rmse:0.301356

Partial score of fold 0 is: 0.8467692297645815
[0]	train-rmse:6.81492	val-rmse:6.76121
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[100]	train-rmse:2.51308	val-rmse:2.48983
[200]	train-rmse:0.955453	val-rmse:0.952952
[300]	train-rmse:0.40917	val-rmse:0.447929
[400]	train-rmse:0.236262	val-rmse:0.325792
[500]	train-rmse:0.187546	val-rmse:0.306565

In [195]:
nn_model = Nn_Model(train, test, features, categoricals=categoricals)

(10000, 91)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 200)               18400     
_________________________________________________________________
layer_normalization (LayerNo (None, 200)               400       
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               20100     
_________________________________________________________________
layer_normalization_1 (Layer (None, 100)               200       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_____________________________

7872/8000 [============================>.] - ETA: 0s - loss: 0.9670
Epoch 00024: val_loss improved from 0.13094 to 0.12661, saving model to nn_model.w8
8000/8000 [==============================] - 2s 252us/sample - loss: 0.9693 - val_loss: 0.1266
Epoch 25/100
7840/8000 [============================>.] - ETA: 0s - loss: 0.9647
Epoch 00025: val_loss did not improve from 0.12661
8000/8000 [==============================] - 2s 250us/sample - loss: 0.9641 - val_loss: 0.1289
Epoch 26/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.9243
Epoch 00026: val_loss did not improve from 0.12661
8000/8000 [==============================] - 2s 248us/sample - loss: 0.9217 - val_loss: 0.1297
Epoch 27/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.9248
Epoch 00027: val_loss did not improve from 0.12661
8000/8000 [==============================] - 2s 249us/sample - loss: 0.9233 - val_loss: 0.1318
Epoch 28/100
7872/8000 [============================>.] - ETA: 0s - loss

Epoch 60/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.5683
Epoch 00060: val_loss did not improve from 0.11193
8000/8000 [==============================] - 2s 248us/sample - loss: 0.5671 - val_loss: 0.1292
Epoch 61/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.5599
Epoch 00061: val_loss did not improve from 0.11193
8000/8000 [==============================] - 2s 249us/sample - loss: 0.5605 - val_loss: 0.1319
Epoch 62/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.5469
Epoch 00062: val_loss improved from 0.11193 to 0.11128, saving model to nn_model.w8
8000/8000 [==============================] - 2s 253us/sample - loss: 0.5477 - val_loss: 0.1113
Epoch 63/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.5599
Epoch 00063: val_loss did not improve from 0.11128
8000/8000 [==============================] - 2s 252us/sample - loss: 0.5597 - val_loss: 0.1143
Epoch 64/100
7872/8000 [============================>.] - E

Epoch 96/100
7840/8000 [============================>.] - ETA: 0s - loss: 0.3694
Epoch 00096: val_loss did not improve from 0.10590
8000/8000 [==============================] - 2s 250us/sample - loss: 0.3672 - val_loss: 0.1122
Epoch 97/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.3506
Epoch 00097: val_loss did not improve from 0.10590
8000/8000 [==============================] - 2s 248us/sample - loss: 0.3504 - val_loss: 0.1096
Epoch 98/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.3395
Epoch 00098: val_loss improved from 0.10590 to 0.10539, saving model to nn_model.w8
8000/8000 [==============================] - 2s 251us/sample - loss: 0.3413 - val_loss: 0.1054
Epoch 99/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.3352
Epoch 00099: val_loss did not improve from 0.10539
8000/8000 [==============================] - 2s 249us/sample - loss: 0.3360 - val_loss: 0.1219
Epoch 100/100
7872/8000 [============================>.] - 

Epoch 21/100
7808/8000 [============================>.] - ETA: 0s - loss: 1.0479
Epoch 00021: val_loss did not improve from 0.13224
8000/8000 [==============================] - 2s 256us/sample - loss: 1.0524 - val_loss: 0.1443
Epoch 22/100
7872/8000 [============================>.] - ETA: 0s - loss: 1.0138
Epoch 00022: val_loss did not improve from 0.13224
8000/8000 [==============================] - 2s 250us/sample - loss: 1.0145 - val_loss: 0.1403
Epoch 23/100
7936/8000 [============================>.] - ETA: 0s - loss: 1.0152
Epoch 00023: val_loss did not improve from 0.13224
8000/8000 [==============================] - 2s 274us/sample - loss: 1.0149 - val_loss: 0.1945
Epoch 24/100
7776/8000 [============================>.] - ETA: 0s - loss: 0.9991
Epoch 00024: val_loss improved from 0.13224 to 0.12318, saving model to nn_model.w8
8000/8000 [==============================] - 2s 260us/sample - loss: 1.0028 - val_loss: 0.1232
Epoch 25/100
7872/8000 [============================>.] - E

...

Epoch 49/100
7776/8000 [============================>.] - ETA: 0s - loss: 0.6952
Epoch 00049: val_loss did not improve from 0.11421
8000/8000 [==============================] - 2s 268us/sample - loss: 0.6986 - val_loss: 0.1263
Epoch 50/100
7776/8000 [============================>.] - ETA: 0s - loss: 0.6796
Epoch 00050: val_loss did not improve from 0.11421
8000/8000 [==============================] - 2s 271us/sample - loss: 0.6781 - val_loss: 0.1176
Epoch 51/100
7904/8000 [============================>.] - ETA: 0s - loss: 0.6702
Epoch 00051: val_loss did not improve from 0.11421
8000/8000 [==============================] - 2s 272us/sample - loss: 0.6687 - val_loss: 0.1327
Epoch 52/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.6688
Epoch 00052: val_loss did not improve from 0.11421
8000/8000 [==============================] - 2s 259us/sample - loss: 0.6696 - val_loss: 0.1281
Epoch 53/100
7840/8000 [============================>.] - ETA: 0s - loss: 0.6532
Epoch 00053

Epoch 85/100
7936/8000 [============================>.] - ETA: 0s - loss: 0.4285
Epoch 00085: val_loss did not improve from 0.10218
8000/8000 [==============================] - 2s 262us/sample - loss: 0.4285 - val_loss: 0.1027
Epoch 86/100
7840/8000 [============================>.] - ETA: 0s - loss: 0.4157
Epoch 00086: val_loss did not improve from 0.10218
8000/8000 [==============================] - 2s 265us/sample - loss: 0.4155 - val_loss: 0.1049
Epoch 87/100
7936/8000 [============================>.] - ETA: 0s - loss: 0.4102
Epoch 00087: val_loss did not improve from 0.10218
8000/8000 [==============================] - 3s 330us/sample - loss: 0.4085 - val_loss: 0.1118
Epoch 88/100
7936/8000 [============================>.] - ETA: 0s - loss: 0.4235
Epoch 00088: val_loss did not improve from 0.10218
8000/8000 [==============================] - 3s 393us/sample - loss: 0.4231 - val_loss: 0.1310
Epoch 89/100
7808/8000 [============================>.] - ETA: 0s - loss: 0.4057
Epoch 00089

Epoch 11/100
7968/8000 [============================>.] - ETA: 0s - loss: 1.2733
Epoch 00011: val_loss improved from 0.17766 to 0.16493, saving model to nn_model.w8
8000/8000 [==============================] - 2s 310us/sample - loss: 1.2745 - val_loss: 0.1649
Epoch 12/100
7872/8000 [============================>.] - ETA: 0s - loss: 1.2267
Epoch 00012: val_loss improved from 0.16493 to 0.15891, saving model to nn_model.w8
8000/8000 [==============================] - 2s 302us/sample - loss: 1.2244 - val_loss: 0.1589
Epoch 13/100
7840/8000 [============================>.] - ETA: 0s - loss: 1.1919
Epoch 00013: val_loss improved from 0.15891 to 0.15212, saving model to nn_model.w8
8000/8000 [==============================] - 3s 352us/sample - loss: 1.1941 - val_loss: 0.1521
Epoch 14/100
7904/8000 [============================>.] - ETA: 0s - loss: 1.2281
Epoch 00014: val_loss improved from 0.15212 to 0.13874, saving model to nn_model.w8
8000/8000 [==============================] - 4s 472us/s

Epoch 46/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.7342
Epoch 00046: val_loss did not improve from 0.11576
8000/8000 [==============================] - 2s 284us/sample - loss: 0.7335 - val_loss: 0.1220
Epoch 47/100
7840/8000 [============================>.] - ETA: 0s - loss: 0.7174
Epoch 00047: val_loss did not improve from 0.11576
8000/8000 [==============================] - 2s 259us/sample - loss: 0.7160 - val_loss: 0.1457
Epoch 48/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.7032
Epoch 00048: val_loss did not improve from 0.11576
8000/8000 [==============================] - 2s 256us/sample - loss: 0.7017 - val_loss: 0.1332
Epoch 49/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.7056
Epoch 00049: val_loss did not improve from 0.11576
8000/8000 [==============================] - 2s 256us/sample - loss: 0.7066 - val_loss: 0.1284
Epoch 50/100
7872/8000 [============================>.] - ETA: 0s - loss: 0.6828
Epoch 00050

Epoch 82/100
7808/8000 [============================>.] - ETA: 0s - loss: 0.4617
Epoch 00082: val_loss did not improve from 0.10809
8000/8000 [==============================] - 2s 295us/sample - loss: 0.4627 - val_loss: 0.1626
Epoch 83/100
7936/8000 [============================>.] - ETA: 0s - loss: 0.4379
Epoch 00083: val_loss did not improve from 0.10809
8000/8000 [==============================] - 2s 294us/sample - loss: 0.4391 - val_loss: 0.1110
Epoch 84/100
7776/8000 [============================>.] - ETA: 0s - loss: 0.4430
Epoch 00084: val_loss did not improve from 0.10809
8000/8000 [==============================] - 2s 303us/sample - loss: 0.4430 - val_loss: 0.1264
Epoch 85/100
7968/8000 [============================>.] - ETA: 0s - loss: 0.4095
Epoch 00085: val_loss did not improve from 0.10809
8000/8000 [==============================] - 3s 322us/sample - loss: 0.4091 - val_loss: 0.1269
Epoch 86/100
7904/8000 [============================>.] - ETA: 0s - loss: 0.4241
Epoch 00086

In [202]:
cat_model = Catb_Model(train, test, features, categoricals=categoricals)

0:	learn: 1.4458413	test: 1.4498946	best: 1.4498946 (0)	total: 82ms	remaining: 6m 49s
100:	learn: 0.3231805	test: 0.3347196	best: 0.3347196 (100)	total: 2.92s	remaining: 2m 21s
200:	learn: 0.2831512	test: 0.3052537	best: 0.3052537 (200)	total: 5.7s	remaining: 2m 16s
300:	learn: 0.2725894	test: 0.3025311	best: 0.3025311 (300)	total: 8.46s	remaining: 2m 12s
400:	learn: 0.2650571	test: 0.3014985	best: 0.3014894 (399)	total: 11.3s	remaining: 2m 9s
500:	learn: 0.2581703	test: 0.3010022	best: 0.3009657 (489)	total: 14.1s	remaining: 2m 6s
600:	learn: 0.2519516	test: 0.3009203	best: 0.3009153 (595)	total: 16.9s	remaining: 2m 3s
700:	learn: 0.2458542	test: 0.3010665	best: 0.3009153 (595)	total: 19.6s	remaining: 2m
800:	learn: 0.2404267	test: 0.3013285	best: 0.3009153 (595)	total: 22.4s	remaining: 1m 57s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3009152921
bestIteration = 595

Shrink model to first 596 iterations.
Partial score of fold 0 is: 0.7647785403869578
0:	learn

# Predict test data

In [197]:
lgb_model.y_pred
#xgb_model.y_pred
#cat_model.y_pred
#nn_model.y_pred

array([9.79380334, 9.9066823 , 9.90203349, ..., 6.07229412, 6.59683023,
       5.06715628])

In [199]:
np.round(lgb_model.y_pred)
#xgb_model.y_pred
#cat_model.y_pred
#nn_model.y_pred

array([10., 10., 10., ...,  6.,  7.,  5.])